# Delaunay-Watershed for segmentation masks compression

In this notebook:
- We present how to compress segmentation masks into a multimaterial mesh. Here we apply it on nuclei segmentation.
- We show that we can recompute the mask images from the mesh.

In this example, the compressed data is 50 times smaller than the original data.

# 0) Install Delaunay-Watershed

In [ ]:
pip install "dw3d"

## 1) Download nuclei data from Stardist 

In [ ]:
from glob import glob

import numpy as np
from csbdeep.utils import Path, download_and_extract_zip_file

download_and_extract_zip_file(
    url       = "https://github.com/stardist/stardist/releases/download/0.3.0/demo3D.zip",
    targetdir = "../data",
    verbose   = 1)


X = sorted(glob("../data/train/images/*.tif"))
Y = sorted(glob("../data/train/masks/*.tif"))
assert all(Path(x).name==Path(y).name for x,y in zip(X,Y, strict=False))

## 2) Compute multimaterial mesh with Delaunay Watershed

In [2]:
from dw3d import GeometryReconstruction3D
from dw3d.mask_reconstruction import reconstruct_mask_from_dict

In [6]:
import skimage.io as io
from scipy.ndimage import zoom

dist = 2
i = 2
segmented_image_reconstructed = zoom(io.imread(Y[i]), [2,1,1],order = 0)
image = zoom(io.imread(X[i]), [2,1,1],order = 1)

reconstruction = GeometryReconstruction3D(segmented_image_reconstructed,min_dist = dist,print_info=False,original_image = image)
# from dw3d.viewing import plot_in_napari
#v = plot_in_napari(DW, add_mesh=True)

## 3) Reconstruct segmentation masks from the mesh. Visualize in napari for comparison

In [7]:
reconstruction.export_compressed_segmentation("Dict_mask.npy")

In [8]:
segmented_image_reconstructed = reconstruct_mask_from_dict("Dict_mask.npy")

In [ ]:
pip install napari[all]

In [ ]:
import napari

v = napari.view_image(segmented_image_reconstructed)
v.add_image(reconstruction.segmented_image)

## 4) Compare files size: mesh VS masks

In [ ]:

npdict = np.load("Dict_mask.npy",allow_pickle=True).item()
dict_memory_footprint = npdict["Verts"].nbytes + npdict["Faces"].nbytes + npdict["seeds"].nbytes + npdict["image_shape"].nbytes
print("Size of Delaunay-Watershed data:",dict_memory_footprint,"bytes")
print("Size of the original segmentation image:",segmented_image_reconstructed.nbytes,"bytes")
print("Relative size:", dict_memory_footprint/segmented_image_reconstructed.nbytes)